In [1]:
pwd

'/Users/aidasaglinskas/Desktop/BC-FacExpr-1.2-fMRI-mainExp'

In [2]:
from matplotlib import pyplot as plt
from PIL import Image, ImageEnhance,ImageStat
import skvideo
import skvideo.io
import numpy as np
import os
from tqdm import tqdm
import alignfaces as afa

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
def load_video(fn):
    #fn = 'ID4_h1_1.mp4'
    videodata = skvideo.io.vread(fn)
    return videodata

In [4]:
def save_video(videodata,ofn='outvideo.mp4'):
    skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-codec': 'h264', '-filter:v' : 'fps=29.97'})

In [5]:
files = [os.path.join('./stimuli/',file) for file in os.listdir('./stimuli/') if file.endswith('.mp4')]
files.sort()
files[0:5]

['./stimuli/ID1_d1_1.mp4',
 './stimuli/ID1_d1_2.mp4',
 './stimuli/ID1_d1_3.mp4',
 './stimuli/ID1_d1_4.mp4',
 './stimuli/ID1_f_1.mp4']

In [6]:
def split_into_frames(vid_fn):
    videodata = load_video(vid_fn)
    nframes = videodata.shape[0]
    frame_filenames = []
    for f in range(nframes):
        frame = videodata[f,:,:,:]
        frame_ofn = vid_fn.replace('.mp4','').replace('./stimuli/','./stimuli_pics/')+f'frame-{f}.png'
        frame_filenames.append(frame_ofn)
        Image.fromarray(frame).save(frame_ofn)
    return frame_filenames,videodata

In [7]:
def crop_frame(image_fn):
    import cv2
    trained_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    #image_fn = './stimuli_pics/ID1_d1_1frame-0.png'
    img = cv2.imread(image_fn)
    img2 = cv2.imread(image_fn)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cordinates = trained_data.detectMultiScale(gray_img)
    
    for (x, y, w, h) in face_cordinates:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 0), -2)
        
    img2[img!=0]=0
    
    return img2,img

In [8]:
def save_cropped(frame_filenames,videodata,vid_ofn):
    mask = np.array([crop_frame(frame_filename)[1] for frame_filename in frame_filenames])
    vid_mask = ((mask==0).sum(-1)==3).sum(axis=0)>1    
    videodata_cropped = videodata.copy()
    nframes = videodata_cropped.shape[0]
    for f in range(nframes):
        for c in range(3):
            videodata_cropped[f,:,:,c][~vid_mask]=0
            
    save_video(videodata_cropped,ofn=vid_ofn)

In [11]:
# CROP ALL VIDS
for vid_fn in tqdm(files):
    frame_filenames,videodata = split_into_frames(vid_fn)
    save_cropped(frame_filenames,videodata,vid_fn.replace('./stimuli/','./stimuli_cropped/'))

100%|██████████████████████████████████████████████████████████████████████████████████████████| 145/145 [31:02<00:00, 12.85s/it]


In [ ]:
# Center of mass for the boxes of indidual frames
# 

In [ ]:
# FIRST ATTEMP WITH MOVING BOX
# cropped_vid = np.array([crop_frame(frame_filename)[0] for frame_filename in tqdm(frame_filenames)])
#save_video(cropped_vid,ofn='croped_1_outvideo.mp4')